In [2]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math
import os
from src.utils import *
from src.data_preprocessing import *
from src.data_processing import *
from src.feature_selection import *
from src.modelisation import *

os.chdir('/Users/ismailakrout/Desktop/python/NLP_statapp')

pd.set_option("display.max_columns", None)


In [3]:
# pour ne pas charger le NB avec des warnings 
import warnings 
warnings.filterwarnings('ignore')

## Data PreProcessing

In [4]:
# On lit la base des speechs
# on peut la rendre plus petite pour faire des tests 
df = read_HouseOfCommons(keep_date=False, rd_lines=True, n=1000)

In [5]:
# on ne garde que Labor et Conservative
df = keep_parties(df, ['Lab', 'Con'])

## Data Processing

In [6]:
# On construit la liste des mots jugés non significatifs
stop_words = construct_list_stopwords()

In [7]:
# On cleen les titres des speechs
df['agenda'] = df['agenda'].apply(clean, args=(stop_words,))

In [8]:
# On construit las liste cleen des topics
list_stem_topics = process_list_BigTech_words(topics)


In [9]:
# On filtre le df et on garde uniquement les speechs sur les BigTech
df = keep_Bigtech_speeches(df, list_stem_topics)

In [10]:
df['text'] = df['text'].apply(clean, args=(stop_words,))

In [11]:
# On fait le groupby() par speaker et party par la suite pour ne pas dépasser les 1 000 000 carc 
df = df.groupby(by=['Speaker', 'party']).sum().reset_index()

On va repérer les 100 mots (puis 500 quand on aura les speeches en entier) qui sont le plus utilisés par chaque parti, et noter leur fréquence d'apparition. 

In [12]:
df_freqs_Con = count_freqs(df, 'Con')
df_freqs_Lab = count_freqs(df, 'Lab')

On a tous les mots utilisés par le parti Conservateur sur les sujets définis, ainsi que leurs fréquences d'apparition. On va retenir les fréquences avec lesquelles chaques mots sont utilisés, par chaque parti, afin de déterminer le chi2 de chaque mot. On se restreint pour l'instant aux partis Con et Lab, fortement majoritaires. 

In [13]:
df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)

## feature selection

In [14]:
# On effectue le etst du Qi 2 
df_freqs = test_qi_2(df_freqs)

On a donc ici chaque mot utilisé par les parti sur le sujet délimité, avec le chi2 correspondant. On va se limiter aux 500 premiers mots :

In [15]:
df_freqs = keep_significatif_word(df_freqs, n=500)

## Data Processing Bis

In [16]:
list_of_words = selected_words(df_freqs)

In [17]:
df = construct_df_reg(df, df_freqs, list_of_words)

In [18]:
df = normalize(df, list_of_words)

In [ ]:
df.loc[df['party'] == 'Con', 'party_bool'] = 1
df.loc[df['party'] == 'Lab', 'party_bool'] = 0

/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_30223/3670770691.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[df['party'] == 'Con', 'party_bool'] = 1


# Regressions linéaires

In [ ]:
import statsmodels.api as sm

dico={}
dico['Speaker'] = ['intercept', 'slope_coefficient']

for word in list_of_words:
    x = df['party_bool']
    y = df[f'{word}']

    # adding the constant term
    x = sm.add_constant(x)

    result = sm.OLS(y, x).fit()

    dico[f'{word}'] = [result.params[0], result.params[1]]


In [ ]:
aux = pd.DataFrame(dico)
df = pd.concat([df, aux]).reset_index(drop=True)

In [ ]:
df.drop(columns=['somme'], inplace=True)
df = df[['Speaker', 'party', 'text']+['party_bool'] + list(df.columns)[3:-1]]

In [ ]:
df

,Speaker,party,text,party_bool,hon,friend,rail,network,cours,consid,govern,programm,busi,hub,job,econom,announc,economi,decis,countri,veri,charter,achiev,disabl,deliv,rais,quick,quit,increas,ladi,parti,possibl,european,northern,import,befor,continu,introduc,express,engag,absolut,nato,appreci,railway,shipbuild,packag,rout,estim,industri,negoti,addit,regul,benefit,creat,improv,propos,employ,parliament,respons,labour,court,opportun,argument,uk,valley,cp,upgrad,highspe,grow,hlos,fear,membership,taxpay,judg,ship,energi,climat,origin,execut,skill,produc,rule,unemploy,competit,electrif,aid,bbc,proud,advoc,element,passeng,transport,extrem,potenti,congratul,fiscal,establish,fee,specif,worri,licenc,recognis,motion,judici,amend,remind,nation,sector,awar,citi,assur,fulli,penkridg,happi,tran,convinc,pennin,littl,wider,success,greenhous,gase,smes,workplac,ptes,determin,grate,encourag,greater,effect,write,sovereignti,context,creation,breakthrough,english,baltic,implement,fleet,vital,fewer,input,parent,probabl,prais,fundament,scottish,agenda,welsh,coln,prioriti,census,complement,websit,inform,y,aim,taxfre,assess,birmingham,hs,russian,evalu,sustain,crowd,guardian,limit,enterpris,primaci,signific,guid,develop,unlik,enunci,privat,malaria,sourc,reduc,easier,tackl,thatth,overal,figur,explor,adopt,news,unit,kingdom,primarili,pii,royal,precis,strategi,reform,dither,answer,challeng,sinc,israel,compani,colleagu,broken,isra,secretari,constitu,issu,fund,prime,gaza,civil,chair,asid,explan,leav,deficit,sometim,tori,liberti,insur,refus,speak,realli,nuclear,blockad,privaci,document,weapon,promot,exampl,law,financ,obvious,renew,decid,shadow,capac,duti,refer,claus,union,england,polici,suggest,avail,gentleman,debat,futur,sever,scott,envisag,permit,damag,churchil,toe,els,favour,confus,vindic,matrix,abroad,crisi,regardless,certif,clackmannanshir,geotherm,bencher,protocol,expens,necessarili,recommend,technic,relat,flow,restat,boost,lincolnshir,kimberlyclark,closur,inadvert,keyn,sensibl,legitim,factori,lobbi,buckingham,immigr,bartonuponhumb,themselv,persuad,materi,secret,everyth,approv,perhap,attract,medic,clinician,distinct,client,auxiliari,minor,paper,analysi,centro,leg,leed,manchest,goahead,natur,infrastructur,scope,ineffici,thus,command,relev,becom,mcnulti,ray,compet,overstretch,elsewher,depend,runup,disappoint,afraid,preempt,midland,strengthen,spread,minut,expos,thwart,oblig,thrust,control,evid,commend,balanc,hexham,prioritis,woodhead,reopen,stocksbridg,peniston,appropri,huddersfield,stalybridg,station,slaithwait,marsden,greenfield,mossley,pick,individu,somewhat,actual,denbi,activ,scotland,traumat,spirit,steadfast,arrest,swift,darci,jason,nineyearold,injuri,louisa,eventu,stab,terribl,issueâ,unlimit,âwe,pop,leveson,danger,stick,timet,slip,offici,experi,dure,phase,thorough,itbecaus,projectal,valueformoney,percept,champion,pension,confid,expertis,necessari,shipyard,capabl,solid,procur,celebr,workforc,invest,navi,framework,feasibl,commiss,hmrc,broad,milton,respond,rapid,guidanc,nonsens,sectionâi,deepli,suffragett,shoulder,regist,equiti,beef,capit,pym,outâ,âone,corpor,hampden,motiona,buri,organis,nuttal,defin,count,claimant,morn,othersbut,virtu,section,pursu,perform,hunt,gdphas,compon,investmenta,profess,fiction,complet,implic,onward,microbusi,rid,oneout,onein,internet,tape,overregul,goldplat,backyard,assert,suprem,ad,statesâth,pool,progress,advanc,insert,impos,platform,caveat,condemn,requir,inspir,brilliant,hitchin,priori,headteach,edward,geraint,alleg,democrat,ultim,embark,accountabilityand,parliamentari,lewi,lock,carrolltyp,âthe,phraseolog,recal,extremist,length,exclus,lamb,net,bed,malari,inconsist,background,nutshel,ill,reconcil,prepar,die,âfiscal
0,Alan Johnson,Lab,"[pleasur, hon, learn, kensington, sir, malcolm...",0.0,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [ ]:
df.to_csv('01. output/df_500_with_slope_coef.csv')